In [1]:
include("../interface/utilities/boilerplate.jl")

In [2]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


# Model parameters

In [2]:
parameters = (
    ρₒ = 1.0,  # reference density
    cₛ = 1e-2, # sound speed
    ℓᵐ = 10,   # jet thickness, (larger is thinner)
    ℓ  = 20,   # perturbation thickness, (larger is thinner)
    m  = 2,    # number of sign changes on equator for perturbation
    ϕᵖ = π/2 * 0.05, # of centerdness of perturbation
    ϵ  = 0.3,  # perturbation amplitude
    vˢ = 5e-4, # velocity scale
    α  = 2e-4,
    Ω  = 1e-3,
);

# Domain setup

In [3]:
domain = SphericalShell(radius = 1.0, height = 0.2);
grid = DiscretizedDomain(
    domain;
    elements              = (vertical = 1, horizontal = 8),
    polynomial_order      = (vertical = 0, horizontal = 3),
    overintegration_order = (vertical = 1, horizontal = 1),
);

# Physics setup

In [4]:
physics = Physics(
    orientation = SphericalOrientation(),
    advection   = NonLinearAdvection(),
    coriolis    = DeepShellCoriolis{Float64}(Ω = parameters.Ω),
    gravity     = Buoyancy{Float64}(α = parameters.α, g = 0.0),
    eos         = BarotropicFluid{Float64}(ρₒ = parameters.ρₒ, cₛ = parameters.cₛ),
);

# Initial conditions

In [5]:
uᵐ(𝒫, λ, ϕ, r) =  𝒫.ℓᵐ * sech(𝒫.ℓᵐ * ϕ)^2 
vᵐ(𝒫, λ, ϕ, r) =  0.0
hᵐ(𝒫, λ, ϕ, r) =  0.0

u1(𝒫, λ, ϕ, r) =  𝒫.ℓ * 2 * (ϕ - 𝒫.ϕᵖ)* exp(-𝒫.ℓ * (ϕ - 𝒫.ϕᵖ)^2) * cos(ϕ) * cos(2 * (ϕ - 𝒫.ϕᵖ)) * sin(𝒫.m * λ)
u2(𝒫, λ, ϕ, r) =  exp(-𝒫.ℓ * (ϕ - 𝒫.ϕᵖ)^2) * sin(ϕ) * cos(2 * (ϕ - 𝒫.ϕᵖ)) * sin(𝒫.m * λ)
u3(𝒫, λ, ϕ, r) =  2*exp(-𝒫.ℓ * (ϕ - 𝒫.ϕᵖ)^2) * cos(ϕ) * sin(2 * (ϕ - 𝒫.ϕᵖ)) * sin(𝒫.m * λ)
uᵖ(𝒫, λ, ϕ, r) =  u1(𝒫, λ, ϕ, r) + u2(𝒫, λ, ϕ, r) + u3(𝒫, λ, ϕ, r)
vᵖ(𝒫, λ, ϕ, r) =  𝒫.m * exp(-𝒫.ℓ * (ϕ - 𝒫.ϕᵖ)^2) * cos(2 * (ϕ - 𝒫.ϕᵖ)) * cos(𝒫.m * λ)
hᵖ(𝒫, λ, ϕ, r) =  0.0

ρ₀(𝒫, λ, ϕ, r)    = 𝒫.ρₒ 
ρuʳᵃᵈ(𝒫, λ, ϕ, r) = 0
ρuˡᵃᵗ(𝒫, λ, ϕ, r) = 𝒫.vˢ * ρ₀(𝒫, λ, ϕ, r) * (𝒫.ϵ * vᵖ(𝒫, λ, ϕ, r))
ρuˡᵒⁿ(𝒫, λ, ϕ, r) = 𝒫.vˢ * ρ₀(𝒫, λ, ϕ, r) * (uᵐ(𝒫, λ, ϕ, r) + 𝒫.ϵ * uᵖ(𝒫, λ, ϕ, r))
ρθ₀(𝒫, λ, ϕ, r) = ρ₀(𝒫, λ, ϕ, r) * tanh(𝒫.ℓᵐ * ϕ);

In [6]:
ρ₀ᶜᵃʳᵗ(𝒫, x...)  = ρ₀(𝒫, lon(x...), lat(x...), rad(x...))
ρu⃗₀ᶜᵃʳᵗ(𝒫, x...) = (   ρuʳᵃᵈ(𝒫, lon(x...), lat(x...), rad(x...)) * r̂(x...) 
                     + ρuˡᵃᵗ(𝒫, lon(x...), lat(x...), rad(x...)) * ϕ̂(x...)
                     + ρuˡᵒⁿ(𝒫, lon(x...), lat(x...), rad(x...)) * λ̂(x...) ) 
ρθ₀ᶜᵃʳᵗ(𝒫, x...) = ρθ₀(𝒫, lon(x...), lat(x...), rad(x...))

ρθ₀ᶜᵃʳᵗ (generic function with 1 method)

# Boundary conditions

In [7]:
bcs = (
    bottom = (ρu = Impenetrable(FreeSlip()), ρθ = Insulating()),
    top =    (ρu = Impenetrable(FreeSlip()), ρθ = Insulating()),
);

# Model setup

In [8]:
model = ModelSetup(
    physics = physics,
    boundary_conditions = bcs,
    initial_conditions = (ρ = ρ₀ᶜᵃʳᵗ, ρu = ρu⃗₀ᶜᵃʳᵗ, ρθ = ρθ₀ᶜᵃʳᵗ),
    numerics = (grid = grid, flux = RoeNumericalFlux()),
    parameters = parameters,
);

# Time stepping and callbacks

In [9]:
Δt          = min_node_distance(grid.numerical) / parameters.cₛ * 0.25
start_time  = 0
end_time    = 8*1600.0
callbacks   = (
    Info(), 
    StateCheck(10), 
    VTKState(iteration = 160, filepath = "./out/"),
);

# Simulation setup

In [10]:
simulation = Simulation(
    model;
    grid = grid,
    timestepper = (method = SSPRK22Heuns, timestep = Δt),
    time        = (start = start_time, finish = end_time),
    callbacks   = callbacks,
);

# Run the things!

In [ ]:
evolve!(simulation);

# SC Start: creating state check callback
# SC Creating state check callback labeled "state" for symbols
# SC ρ
# SC ρu
# SC ρθ
# SC Finish: creating state check callback


┌ Info: doing VTK output
└   outprefix = "./out//mpirank0000_step0000"
┌ Info: doing VTK output
└   outprefix = "./out//mpirank0000_step0001"
┌ Info: doing VTK output
└   outprefix = "./out//mpirank0000_step0002"
┌ Info: doing VTK output
└   outprefix = "./out//mpirank0000_step0003"
